In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import copy
from math import sqrt,ceil

from rope.base.oi.soa import analyze_module
from scipy.stats import linregress
import libraries as lib

In [19]:
directory = "../../../OneDrive - HvA/Logfiles/SOLAR/2023/20230615_testvaart_Amstel/0014 - 20230615T131248 3h14m8s 22.186km Duivendrecht"
file = "1.csv"

In [21]:
data_file = lib.DataInladen(f'{directory}/{file}')
data_file

08|04
   1  331.7514  08|04   $GPRMC  111248.000  A  5219.8936  N  00453.9587  E  \
0  1  332.7614  08|04   $GPRMC    111249.0  A  5219.8936  N    453.9587  E   
1  1  333.7367  08|04   $GPRMC    111250.0  A  5219.8936  N    453.9587  E   
2  1  334.7538  08|04   $GPRMC    111251.0  A  5219.8936  N    453.9587  E   
3  1  335.7802  08|04   $GPRMC    111252.0  A  5219.8936  N    453.9587  E   
4  1  336.7533  08|04   $GPRMC    111253.0  A  5219.8936  N    453.9587  E   

   0.0  221.6  150623  Unnamed: 13  Unnamed: 14 A.1  000111001011  0.00  
0  0.0  221.6  150623          NaN          NaN   A     111001011   0.0  
1  0.0  221.6  150623          NaN          NaN   A     111001011   0.0  
2  0.0  221.6  150623          NaN          NaN   A     111001011   0.0  
3  0.0  221.6  150623          NaN          NaN   A     111001011   0.0  
4  0.0  221.6  150623          NaN          NaN   A     111001011   0.0  
['Datalogger port', 'Dataloggertijd, in s', 'Format header (>08|04)', 'NMEA type(

,Datalogger port,"Dataloggertijd, in s",Format header (>08|04),NMEA type($GPRMC),Tijd,Status,Latitude,Noord of Zuid (N/S),Longitude,Oost of West (E/W),Snelheid over de grond in km/h,Richting van de snelheid,Datum,Magnetische variatie,Magnetische directie,Mode indicator (A),Data in balancers,Snelheid tenopzichte van het water in km/u
0,1,332.7614,08|04,$GPRMC,111249.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
1,1,333.7367,08|04,$GPRMC,111250.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
2,1,334.7538,08|04,$GPRMC,111251.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
3,1,335.7802,08|04,$GPRMC,111252.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
4,1,336.7533,08|04,$GPRMC,111253.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11643,1,11975.8896,08|04,$GPRMC,142652.0,A,5219.9052,N,453.9402,E,0.0,1.9,150623,NaN,NaN,D,111001011,0.0
11644,1,11976.8664,08|04,$GPRMC,142653.0,A,5219.9052,N,453.9402,E,0.0,1.9,150623,NaN,NaN,D,111001011,0.0
11645,1,11977.8414,08|04,$GPRMC,142654.0,A,5219.9052,N,453.9402,E,0.0,1.9,150623,NaN,NaN,D,111001011,0.0
11646,1,11978.8763,08|04,$GPRMC,142655.0,A,5219.9052,N,453.9402,E,0.0,1.9,150623,NaN,NaN,D,111001011,0.0


In [27]:
xas = 'Latitude'
yas = 'Longitude'
zas = 'Snelheid over de grond in km/h'
# Filtering the data so it's only the laps we want to analyze
# First filter so that the data without GPS data is removed
data_file_filtered = data_file[data_file[xas] != 0]
data_file_filtered = data_file[data_file[yas] != 0]

# Now filter out the beginning and end of the data, by removing the first and last 10% of the data
from math import ceil
#data_file
#data_file = data_file.iloc[ceil(len(data_file)*0.215):ceil(len(data_file)*0.267)]

# Display the filtered data
data_file_filtered

,Datalogger port,"Dataloggertijd, in s",Format header (>08|04),NMEA type($GPRMC),Tijd,Status,Latitude,Noord of Zuid (N/S),Longitude,Oost of West (E/W),Snelheid over de grond in km/h,Richting van de snelheid,Datum,Magnetische variatie,Magnetische directie,Mode indicator (A),Data in balancers,Snelheid tenopzichte van het water in km/u
0,1,332.7614,08|04,$GPRMC,111249.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
1,1,333.7367,08|04,$GPRMC,111250.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
2,1,334.7538,08|04,$GPRMC,111251.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
3,1,335.7802,08|04,$GPRMC,111252.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
4,1,336.7533,08|04,$GPRMC,111253.0,A,5219.8936,N,453.9587,E,0.0,221.6,150623,NaN,NaN,A,111001011,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11643,1,11975.8896,08|04,$GPRMC,142652.0,A,5219.9052,N,453.9402,E,0.0,1.9,150623,NaN,NaN,D,111001011,0.0
11644,1,11976.8664,08|04,$GPRMC,142653.0,A,5219.9052,N,453.9402,E,0.0,1.9,150623,NaN,NaN,D,111001011,0.0
11645,1,11977.8414,08|04,$GPRMC,142654.0,A,5219.9052,N,453.9402,E,0.0,1.9,150623,NaN,NaN,D,111001011,0.0
11646,1,11978.8763,08|04,$GPRMC,142655.0,A,5219.9052,N,453.9402,E,0.0,1.9,150623,NaN,NaN,D,111001011,0.0


In [30]:
%matplotlib qt
'Plotting the race'
plt.figure(figsize=(6,6))
plt.scatter(data_file[xas], data_file[yas], c=data_file[zas], cmap='viridis')
plt.title(xas + ' vs ' + yas + ' with ' + zas)
plt.xlabel(xas)
plt.ylabel(yas)

# Adding a colorbar with a label for the speed in km/h
cbar = plt.colorbar()
cbar.set_label(zas)
plt.grid()
plt.show()


In [9]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract data for plotting
latitudes = data_file['Latitude'].values
longitudes = data_file['Longitude'].values
speeds = data_file['Snelheid over de grond in km/h'].values

# Plotting the race with a 3D scatter plot
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot with height representing speed
scatter = ax.scatter(latitudes, longitudes, speeds, c=speeds, cmap='viridis', s=10)

# Adding labels and title
ax.set_title('GPS data vs speed')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
ax.set_zlabel('Speed (km/h)')

# Adding a colorbar with a label for the speed in km/h
cbar = plt.colorbar(scatter, ax=ax, pad=0.1)
cbar.set_label('Speed (km/h)')

plt.show()

In [10]:
def plotten(xas,yas, zas = None):
    return
